<a href="https://colab.research.google.com/github/mystlee/2024_CSU_AI/blob/main/chapter8/example_speaker_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install speechbrain torchaudio huggingface_hub matplotlib datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import pipeline
from IPython.display import Audio
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

In [ ]:
def map_to_array(example):
    speech, _ = librosa.load(example["file"],
                             sr = 16000,
                             mono = True)
    example["speech"] = speech
    return example

dataset = load_dataset("anton-l/superb_demo", "si", split = "test")
dataset = dataset.map(map_to_array)

model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

inputs = feature_extractor(dataset[:2]["speech"],
                           sampling_rate = 16000,
                           padding = True,
                           return_tensors = "pt")

logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim = -1)
labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]

Repo card metadata block was not found. Setting CardData to empty.
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
# 누군지는 몰라도 id3번과 id4번의 목소리 인거 같
print(labels)

['id10003', 'id10004']


In [ ]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier, SpeakerRecognition
from google.colab import files

uploaded = files.upload()
file_paths = []
for filename in uploaded.keys():
    file_paths.append(filename)
print(file_paths)

Saving spk1_snt5.wav to spk1_snt5.wav
Saving spk1_snt6.wav to spk1_snt6.wav
Saving spk2_snt1.wav to spk2_snt1 (2).wav
['spk1_snt5.wav', 'spk1_snt6.wav', 'spk2_snt1 (2).wav']


In [ ]:
print("file 1")
display(Audio(file_paths[0]))
print("\nfile 2")
display(Audio(file_paths[1]))
print("\nfile 3")
display(Audio(file_paths[2]))

file 1



file 2



file 3


In [ ]:
model = SpeakerRecognition.from_hparams(source = "speechbrain/spkrec-ecapa-voxceleb",
                                        savedir = "pretrained_models/spkrec-ecapa-voxceleb")
# 같은 speaker
same_score, same_prediction = model.verify_files(file_paths[0], file_paths[1])
# 다른 speaker
diff_score, diff_prediction = model.verify_files(file_paths[0], file_paths[2])
print("같은 화자")
print("유사도: {:2.2f}".format(same_score.numpy()[0]))
print("예측: {}".format(same_prediction.numpy()[0]))
print("다른 화자")
print("유사도: {:2.2f}".format(diff_score.numpy()[0]))
print("예측: {}".format(diff_prediction.numpy()[0]))

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

같은 화자
유사도: 0.61
예측: True
다른 화자
유사도: 0.10
예측: False
